# Step15, 복잡한 계산 그래프(이론편)

지금까지는 한 줄로 늘어선 계산 그래프를 다뤘다.  
하지만 변수와 함수가 꼭 한 줄로 연결되리라는 법은 없다.  
같은 변수를 반복해서 사용하거나 여러 변수를 입력받는 함수를 사용하는 계산을 할 수 있다.  
이를 통해 더 복잡한 연결을 만들 수 있다.

하지만 지금의 DeZeo는 이런 계산의 미분을 하지 못한다.즉, 이런 복잡한 연결의 역전파를 제대로 할 수 없다.

**NOTE_** 그래프의 '연결된 형태' : _위상_(topology)  
다양한 위상의 계산 그래프에 대응하는 것이 목표  
어떤 모양으로 연결된 계산 그래프라도 제대로 미분할 수 있도록 지금부터 새로운 아이디어를 도입

## 15.1 역전파의 올바른 순서
책에 그림과 함께 아주 잘 설명되어있다. 책 참고하기 

## 15.2 현재의 DeZero

In [1]:
class Variable:
    
    # ...생략.... 

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()                             # 주목
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:   
                    funcs.append(x.creator)             # 주목


while 블록 마지막줄에 처리할 함수의 후보를 func 리스트의 끝에 추가  : func.append(x.creator)  
다음에 처리할 함수를 그 리스트의 끝에서 꺼낸다 : funcs.pop()  

책 설명 

## 15.3 함수 우선순위

funcs 리스트에는 다음에 처리할 함수의 '후보'들이 들어 있다.  
지금까지는 아무생각없이 '마지막' 원소만 꺼냈다.  
이 문제를 해결하기 위해서는 함수에 '우선순위'를 줄 수 있어야 한다.  

1. 주어진 계산 그래프를 '분석'하여 알아내는 방법
    - 위상 정렬 알고리즘을 사용하면 노드의 연결 방법을 기초로 노드들을 정렬할 수 있다.
    - 이 '정렬 순서'가 우선순위가 된다.

2. 우리는 일반적인 계산(순전파) 때 '함수'가 '변수'를 만들어내는 과정을 '목격'하고 있다.  
    - 즉, 어떤 함수가 어떤 변수를 만들어내는가 하는 '관계'를 이미 목격하고 있다.  
    - 이 관계를 기준으로 함수와 변수의 '세대(generation)'을 기록할 수 있다.  
    - 이 '세대'가 우선순위가 된다.

# Step16, 복잡한 계산 그래프(구현 편)

1. 순전파 시 '세대'를 설정하는 부분
2. 역전파 시 최근 세대의 함수부터 꺼낸다.  

이렇게 하면 아무리 복잡한 계산 그래프라도 올바른 순서로 역전파가 이루어진다.

## 16.1 세대 추가 

인스턴스 변수 generation을 Variable 클래스와 Function 클래스에 추가.  
몇 번째 '세대'의 함수(혹은 변수)인지 나타내는 변수이다.

In [3]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        
        self.data = data 
        self.grad = None        # 미분 변수
        self.creator = None     # 변수의 창조자를 기록하는 변수
        self.generation = 0     # 세대를 기록하는 변수
    
    def set_creator(self,func):
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)
    
    # ..... 생략 ..... 

Variable 클래스  
Variable 클래스는 generation을 0으로 초기화한다.  
set_creator 메서드가 호출될 때 부모 함수의 세대보다 1 큰 값을 설정한다.
f.generation이 2인 함수에서 만들어진 변수인 y의 generation은 3이된다.

Function 클래스  
Function 클래스의 generation은 입력 변수와 같은 값으로 설정  
입력 변수의 generation이 4라면 함수의 generation도 4가 된다.  
입력 변수가 둘 이상이면 가장 큰 generation의 수를 선택한다.  
    - 예를 들어 입력변수 2개의 generation이 각각 3과 4라면 함수의 generation은 4이다.

In [5]:
class Function(object):
    def __call__(self,*inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)

        self.inputs = inputs 
        self.outputs = outputs

        return outputs if len(outputs) > 1 else outputs[0]

    # ..... 생략 .....

# 최종 코드

In [43]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)
    
    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = [self.creator]                      # 함수들을 담는 리스트 
        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output.grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)



def as_array(x):
    if np.isscalar(x):
        return np.array(x)                                  # 입력이 스칼라인 경우 ndarray 인스턴스로 변화해줌
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]                       # 가변길이 인수를 다루기위해, 변수를 리스트에 담아 취급
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]       # 가변길이 입력이므로 가변길이 출력을 리스트로 담는다

        self.generation = max([x.generation for x in inputs])   # inputs의 generation중에 가장 큰것

        for output in outputs:
            output.set_creator(self)                     # 각각의 output들이 어디 출신 변수인지 정해짐, 자신이 창조자라고 원산지 표시를 함

        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]  # 리스트의 원소가 하나라면 첫번째 원소를 반환한다, 해당 변수를 직접 돌려준다

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    f = Square()
    return f(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

## 16.2 세대 순으로 꺼내기 

일반적인 계산(순전파)을 하면 모든 변수와 함수에 세대가 설정된다.  
이렇게 세대가 설정되어 있으면 역전파 때 함수를 올바른 순서로 꺼낼수 있다.

**NOTE_** Variable 클래스의 backward 메서드안에서는 처리할 함수의 후보들을 funcs 리스트에 보관한다.  
따라서 funcs에서 세대가 큰 함수부터 꺼내게 하면 올바른 순서로 역전파 할 수 있다.

In [44]:
# 함수를 세대 순으로 꺼내는 간단한 실험을 해본다.

generations = [2,0,1,4,2]
funcs = []

for g in generations:
    f = Function()      # 더미 함수 클래스 
    f.generation = g 
    funcs.append(f)

print([f.generation for f in funcs])

[2, 0, 1, 4, 2]


In [45]:
# 위와 같이 더미 함수를 준비하고 funcs 리스트에 추가한다. 그런다음 이 리스트에서 세대가 가장 큰 함수를 꺼낸다.
funcs.sort(key=lambda x: x.generation)  # 리스트의 원소 x를 x.generation을 키로 사용해 정렬해라
print([f.generation for f in funcs])

f = funcs.pop()                         # 가장 큰 값을 꺼낸다.
print(f.generation)

[0, 1, 2, 2, 4]
4


## 16.3 Variable 클래스의 backward 

달라진 부분 찾기 

In [48]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):                            # 입력받는 데이터가 ndarray 구조가 아니면 오류 발생
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data        # 데이터 선언
        self.grad = None        # 미분값 선언
        self.creator = None     # 이 데이터가 어디출신인지, 어느 공장에서 만들어졌는지 표기
        self.generation = 0     # 세대를 기록하는 변수

    def set_creator(self, func):    # 생성자 = 공장 = 함수
        self.creator = func 
        self.generation = func.generation + 1   # 세대를 기록한다(부모 세대 + 1)

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬

        funcs = []                      # <-- 바뀐부분
        seen_set = set()                # 집합은 중복을 막는다.

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)              # DezZero 함수 리스트를 세대순으로 정렬하는 역할 
                                            # 그결과 funcs.pop()은 자동으로 세대가 가장 큰 DeZero 함수 순으로 꺼낸다. 

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output.grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            gxs = f.backward(*gys)                          # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
            if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                if x.grad is None:
                    x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.


그동안 'DeZero 함수'를 리스트에 추가할 때 funcs.append(f)를 호출했다.  
대신 add_func() 함수를 호출하도록 변경  
add_func() 함수가 DeZero 함수 리스트를 세대 순으로 정렬하는 역할 
그 결과 funcs.pop()은 자동으로 세대가 가장 큰 순서대로 함수를 꺼낸다. 

senn_set = set()에서 집합을 이용하고 있다.  
funcs 리스트에 같은 함수를 중복 추가하는 일을 막기 위해서이다.  
때문에 함수의 backward 메서드가 잘못되어 여러 번 불리는 일은 발생하지 않는다.

## 16.4 동작 확인 
세대가 큰 함수부터 꺼낼 수 있게 되었다.  
아무리 복잡한 계산 그래프의 역전파도 올바른 순서로 진행할 수 있다.

In [49]:
# a = x^2
# y = a^2 + a^2 --> (x^2)^2 + (x^2)^2 = 2x^4
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a),square(a))
y.backward()
# y' = 8x^3

print(y.data)
print(x.grad)

32.0
64.0
